In [2]:
!pip install mediapipe opencv-python scikit-learn xgboost numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.6 which is incompatible.


In [3]:
!pip uninstall -y mediapipe

Found existing installation: mediapipe 0.10.21
Uninstalling mediapipe-0.10.21:
  Successfully uninstalled mediapipe-0.10.21


In [4]:
!pip install mediapipe==0.10.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.


In [5]:
!pip install --upgrade numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 890.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 68.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.


In [2]:
# Imports
import io
import cv2
import statistics
import numpy as np
import pandas as pd
import mediapipe as mp
from pickle import load
from collections import deque
from google.colab import files
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from IPython.display import display, Image, clear_output

### **Helper Functions**

In [3]:
# We need to center by subtracting the wrist position, followed by averaging with the middle finger tip position
# Wrist = 0, index (x1, y1, z1)
# Middle finger tip = 12, index (x13, y13, z13)

def center_landmarks(df, col):

    '''
    Function to center the coordinates by subtracting from the wrist coordinates. Ignore z coordinates.

    Args:
      col (pandas.series) : The dataframe column we want to center, as a pandas series. Center by subtracting wrist position.

    Returns:
      col (pandas.series) : The centered dataframe column, as a pandas series.
    '''

    col_name = col.name       # Extract column name

    # Check if it is the x or y coordinate first, otherwise neglect the column
    if 'x' in col_name:
      col = (col - df['x1'])

    elif 'y' in col_name:
      col = (col - df['y1'])

    return col



def normalize_landmarks(df, col):

    '''
    Function to normalize the centered coordinates by dividing with the mid-finger tip coordinates. Ignore z coordinates.

    Args:
      col (pandas.series) : The dataframe column we want to normalize, as a pandas series. Normalize by dividing middle finger tip position.

    Returns:
      col (pandas.series) : The normalized dataframe column, as a pandas series.
    '''

    col_name = col.name       # Extract column name

    # Check if it is the x or y coordinate first, otherwise neglect the column
    if 'x' in col_name:
      col = col / df['x13']

    elif 'y' in col_name:
      col = col / df['y13']

    return col

### **Loading Models & Encoders**

In [4]:
# Path of the Landmarks Extraction Model
model_path = 'hand_landmarker.task'

In [5]:
# Generate Feature Names to reconstruct the dataframes for prediction
feature_names = [f'{coord}{i}' for i in range(1, 22) for coord in ['x', 'y', 'z']]

In [6]:
# Load models
with open("/content/XGB0.9762414800389484.pkl", "rb") as f:
    xgb_clf = load(f)

with open("/content/RandomForest_0.9509250243427458.pkl", "rb") as f:
    rf_clf = load(f)

with open("/content/ExtraTrees0.9645569620253165.pkl", "rb") as f:
    et_clf = load(f)

with open("/content/OvR_SVM_RBF_0.9727361246348588.pkl", "rb") as f:
    svm_rbf_clf = load(f)

with open("/content/OvR_SVM_Poly_0.9779941577409932.pkl", "rb") as f:
    svm_poly_clf = load(f)

In [7]:
# Load the saved label encoder (For XGB Model)
with open("label_encoder.pkl", "rb") as f:
    label_encoder = load(f)

### **Testing on Images**

In [ ]:
# Initialize MediaPipe Hand module
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

In [ ]:
# Load the image
image_path = "/content/one.png"
image = cv2.imread(image_path)
if image is None:
    raise ValueError("Error: Could not read the image file.")


# Convert image to RGB (required by MediaPipe)
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


# Process the image to detect hand landmarks
results = hands.process(rgb_image)


if results.multi_hand_landmarks:

    for landmarks in results.multi_hand_landmarks:

        mp_drawing.draw_landmarks(image, landmarks, mp_hands.HAND_CONNECTIONS)

        # Extract landmark coordinates
        landmarks_coords = [[lm.x, lm.y, lm.z] for lm in landmarks.landmark]
        feature_vector = np.array(landmarks_coords).flatten().reshape(1, -1)
        feature_vector = pd.DataFrame([np.array(landmarks_coords).flatten()], columns=feature_names)

        # Scale the input Features
        feature_vector_cpy = feature_vector.copy()
        feature_vector_centered = feature_vector.apply(lambda col : center_landmarks(feature_vector, col), axis=0)
        feature_vector_normalized = feature_vector_centered.apply(lambda col : normalize_landmarks(feature_vector, col), axis=0)


        # Perform gesture classification using voting
        predictions = [ svm_rbf_clf.predict(feature_vector_normalized)[0],
                        svm_poly_clf.predict(feature_vector_normalized)[0],
                        rf_clf.predict(feature_vector_normalized)[0],
                        et_clf.predict(feature_vector_normalized)[0],
                        label_encoder.inverse_transform([xgb_clf.predict(feature_vector_normalized)[0]])[0]
                     ]

        print("Models Predictions:", predictions)

        # Voting mechanism (choose the most common prediction)
        try:
            final_prediction = statistics.mode(predictions)

        except statistics.StatisticsError:
            final_prediction = predictions[-1]  # Choose XGB if there's a tie

        # Display the prediction on the image
        cv2.putText(image, f'Gesture: {final_prediction}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
else:
    print("No hand detected in the image.")

# Save and display the processed image
output_path = "output_image.jpg"
cv2.imwrite(output_path, image)
cv2_imshow(image)


print(f"Processing complete! The output image is saved as '{output_path}'.")

### **Testing on Videos**

In [8]:
# Initialize MediaPipe Hand module
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands()

In [14]:
# Read the uploaded video
video_path = "webcam_capture.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Set up video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_video.mp4', fourcc, fps, (width, height))

# Define a deque (rolling window) to store past predictions
WINDOW_SIZE = 5  # Adjust the window size based on your needs
prediction_history = deque(maxlen=WINDOW_SIZE)

print("Processing video...")

while cap.isOpened():

    ret, frame = cap.read()
    if not ret:
        break

    # Convert to RGB (required for MediaPipe)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process frame using MediaPipe
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:


        for landmarks in results.multi_hand_landmarks:

            mp_drawing.draw_landmarks(frame, landmarks, mp_hands.HAND_CONNECTIONS)

            # Extract landmark coordinates
            landmarks_coords = [[lm.x, lm.y, lm.z] for lm in landmarks.landmark]
            feature_vector = pd.DataFrame([np.array(landmarks_coords).flatten()], columns=feature_names)

            # Scale the input features
            feature_vector_centered = feature_vector.apply(lambda col: center_landmarks(feature_vector, col), axis=0)
            feature_vector_normalized = feature_vector_centered.apply(lambda col: normalize_landmarks(feature_vector, col), axis=0)

            # Perform gesture classification using voting
            predictions = [
                svm_rbf_clf.predict(feature_vector_normalized)[0],
                svm_poly_clf.predict(feature_vector_normalized)[0],
                rf_clf.predict(feature_vector_normalized)[0],
                et_clf.predict(feature_vector_normalized)[0],
                label_encoder.inverse_transform([xgb_clf.predict(feature_vector_normalized)[0]])[0]
            ]

            print('Models Prediction:', predictions)

            # Voting mechanism (choose the most common prediction)
            try:
                final_prediction = statistics.mode(predictions)

            except statistics.StatisticsError:
                final_prediction = predictions[-1]  # Use XGB as a fallback

            # Store the prediction in the rolling window
            prediction_history.append(final_prediction)

            # Compute the stabilized output (mode of last `WINDOW_SIZE` predictions)
            try:
                stabilized_prediction = statistics.mode(prediction_history)
            except statistics.StatisticsError:
                stabilized_prediction = prediction_history[-1]  # Use the latest value as fallback

            # Display the stabilized prediction on the frame
            cv2.putText(frame, f'Gesture: {stabilized_prediction}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Write processed frame to output video
    out.write(frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print("Processing complete! The output video is saved as 'output_video.mp4'.")

Processing video...
Models Prediction: ['four', 'four', 'palm', 'four', 'four']
Models Prediction: ['four', 'four', 'four', 'four', 'four']
Models Prediction: ['four', 'four', 'four', 'four', 'four']
Models Prediction: ['palm', 'four', 'palm', 'four', 'palm']
Models Prediction: ['four', 'four', 'four', 'four', 'four']
Models Prediction: ['four', 'four', 'four', 'four', 'four']
Models Prediction: ['four', 'four', 'four', 'four', 'four']
Models Prediction: ['four', 'four', 'four', 'four', 'four']
Models Prediction: ['four', 'four', 'four', 'four', 'four']
Models Prediction: ['four', 'four', 'four', 'four', 'four']
Models Prediction: ['four', 'four', 'four', 'four', 'four']
Models Prediction: ['four', 'four', 'four', 'four', 'four']
Models Prediction: ['four', 'four', 'four', 'four', 'four']
Models Prediction: ['four', 'four', 'four', 'four', 'four']
Models Prediction: ['four', 'four', 'four', 'four', 'four']
Models Prediction: ['four', 'four', 'four', 'four', 'four']
Models Prediction: [

In [15]:
files.download("output_video.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>